In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
# import geoopt
import manifolds
import layers.hyp_layers as hyp_layers
import utils.math_utils as pmath
import optimizers
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
n_data = torch.ones(100, 2)
x0 = torch.normal(2*n_data, 1)
y0 = torch.zeros(100)
x1 = torch.normal(-2*n_data, 1)
y1 = torch.ones(100)

x = torch.cat((x0, x1), 0).type(torch.FloatTensor)  # FloatTensor = 32-bit floating
y = torch.cat((y0, y1), ).type(torch.LongTensor)    # LongTensor = 64-bit integer

print(x.detach().numpy())
plt.scatter(x.detach().numpy(), y.detach().numpy())
# plt.show()

[[ 1.796129    2.1868322 ]
 [ 1.7100279   2.8140442 ]
 [ 1.235987    2.3387034 ]
 [ 2.1493998   3.2302608 ]
 [ 2.0747974   2.3686128 ]
 [ 0.8994112   3.916121  ]
 [ 1.9367622   0.44875455]
 [ 1.9738387   3.5876145 ]
 [ 4.6340346   2.8736534 ]
 [ 3.152142    1.6565876 ]
 [ 3.1138315   2.207151  ]
 [ 3.734939    2.0592616 ]
 [ 0.9254284   3.0757728 ]
 [ 2.0052185   2.6898935 ]
 [ 2.1260293   1.811782  ]
 [ 2.0144053   1.8061991 ]
 [ 0.8387127   2.1443353 ]
 [ 2.1168306   1.989832  ]
 [ 2.102544    2.3392096 ]
 [ 1.3232805   3.1415274 ]
 [ 1.2104527   1.941552  ]
 [ 2.8299012   1.1343079 ]
 [ 1.4139744   3.0992694 ]
 [ 2.505512    0.7896763 ]
 [ 0.31563783  1.445152  ]
 [ 1.658386    0.17534566]
 [ 1.8758687   2.3231602 ]
 [ 2.3809705   2.725052  ]
 [ 2.020403    1.7318258 ]
 [ 3.1010246   1.7558494 ]
 [ 4.3841953   1.9918698 ]
 [ 0.79744947  4.2572274 ]
 [ 1.0052218   2.3976593 ]
 [ 1.8879421   2.8303652 ]
 [ 1.185845    2.4496427 ]
 [ 0.8573315   3.5953608 ]
 [-0.7238126   0.3117702 ]
 

In [ ]:
"""
args
"""
class config_args():
    def __init__(self):
        self.manifold = 'PoincareBall' # which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall]
        self.dropout = 0.0
        self.bias = 1
        self.c = 1.0
        self.num_layers = 2
        self.cuda = -1 # which cuda device to use (-1 for cpu training)
        self.act = 'relu'
        self.dim = 128 # embedding dimension
        self.task = 'None'
        self.optimizer = 'RiemannianAdam' # which optimizer to use, can be any of [Adam, RiemannianAdam]
        self.lr = 0.01
        self.weight_decay = 0.
    
args = config_args()
args.feat_dim = 1

In [ ]:
class Encoder(nn.Module):
    """
    Encoder abstract class.
    """

    def __init__(self, c):
        super(Encoder, self).__init__()
        self.c = c

    def encode(self, x, adj):
        if self.encode_graph:
            input = (x, adj)
            output, _ = self.layers.forward(input)
        else:
            output = self.layers.forward(x)
        return output

In [ ]:
class HNN(Encoder):
    """
    Hyperbolic Neural Networks.
    """

    def __init__(self, c, args):
        super(HNN, self).__init__(c)
        self.manifold = getattr(manifolds, args.manifold)()
        assert args.num_layers > 1
        dims, acts, _ = hyp_layers.get_dim_act_curv(args)
        hnn_layers = []
        for i in range(len(dims) - 1):
            in_dim, out_dim = dims[i], dims[i + 1]
            act = acts[i]
            hnn_layers.append(
                    hyp_layers.HNNLayer(
                            self.manifold, in_dim, out_dim, self.c, args.dropout, act, args.bias)
            )
        self.layers = nn.Sequential(*hnn_layers)
        self.encode_graph = False

    def encode(self, x, adj):
        x_hyp = self.manifold.proj(self.manifold.expmap0(self.manifold.proj_tan0(x, self.c), c=self.c), c=self.c)
        return super(HNN, self).encode(x_hyp, adj)

In [ ]:
class Decoder(nn.Module):
    """
    Decoder abstract class for node classification tasks.
    """

    def __init__(self, c):
        super(Decoder, self).__init__()
        self.c = c

    def decode(self, x, adj):
        if self.decode_adj:
            input = (x, adj)
            probs, _ = self.cls.forward(input)
        else:
            probs = self.cls.forward(x)
        return probs

In [ ]:
class LinearDecoder(Decoder):
    """
    MLP Decoder for Hyperbolic/Euclidean node classification models.
    """

    def __init__(self, c, args):
        super(LinearDecoder, self).__init__(c)
        self.manifold = getattr(manifolds, args.manifold)()
        self.input_dim = args.dim
        self.output_dim = args.n_classes
        self.bias = args.bias
        self.cls = hyp_layers.Linear(self.input_dim, self.output_dim, args.dropout, lambda x: x, self.bias)
        self.decode_adj = False

    def decode(self, x, adj):
        h = self.manifold.proj_tan0(self.manifold.logmap0(x, c=self.c), c=self.c)
        return super(LinearDecoder, self).decode(h, adj)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}, c={}'.format(
                self.input_dim, self.output_dim, self.bias, self.c
        )

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, args):
        super(BaseModel, self).__init__()
        self.manifold_name = args.manifold
        if args.c is not None:
            self.c = torch.tensor([args.c])
            if not args.cuda == -1:
                self.c = self.c.to(args.device)
        else:
            self.c = nn.Parameter(torch.Tensor([1.]))
        self.manifold = getattr(manifolds, self.manifold_name)()
        if self.manifold.name == 'Hyperboloid':
            args.feat_dim = args.feat_dim + 1
        self.encoder = HNN(self.c, args)
    
    def encode(self, x, adj = None):
        if self.manifold.name == 'Hyperboloid':
            o = torch.zeros_like(x)
            x = torch.cat([o[:, 0:1], x], dim=1)
        h = self.encoder.encode(x, adj)
        return h

    def decode(self, h)
    
    def compute_metrics(self, embeddings, data, split):
        raise NotImplementedError

In [ ]:
model = BaseModel(args)
optimizer = getattr(optimizers, args.optimizer)(params=model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(
#         optimizer,
#         step_size=int(args.lr_reduce_freq),
#         gamma=float(args.gamma)
#     )

In [ ]:
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    embeddings = model.encode(data...)
    loss = F.loss
    loss.backward()
    optimizer.step()
    # lr_scheduler.step()
    
    